# Face Off Analysis

Root question is: "Who does Patrice Bergeron lost faceoffs to most"

From there I'm sure I'll want to explore
- What zone are most face off losses?
- How often is Bergy waved off the dot?
- What are his percentages in each zone?
- How does period and +/- impact faceoff percentage?

But first is something to parse a game file.

In [24]:
# Set up the environment and pull in the game data
import pandas as pd
import json

# Pulled from https://statsapi.web.nhl.com/api/v1/game/{1}/feed/live
filename = 'api_data/game_2019020010.json'

with open(filename) as json_file:
    game_data = json.load(json_file)

## Get Faceoff data

Get the faceoff data from the liveData file and massage it into something more helpful. Basically flatten the Faceoff data.

- [ ] [TODO] Push the PlayID into the index for this DataFrame

In [25]:
plays = json_normalize(game_data['liveData']['plays']['allPlays'])

players_list = plays.loc[lambda r: r['result.event'] == 'Faceoff']['players']

faceoff_list = []

for i, p in players_list.items():
    play_object = { 'winner': None, 'loser': None }
    for pi in p:
        if pi['playerType'] == 'Winner':
            play_object['winner'] = pi['player']['fullName']
        else:
            play_object['loser'] = pi['player']['fullName']
        
    faceoff_list.append(play_object)

faceoff_df = pd.DataFrame.from_dict(faceoff_list)

faceoff_df.head(5)

,winner,loser
0,Patrice Bergeron,Tyler Seguin
1,Mattias Janmark,Sean Kuraly
2,Roope Hintz,Par Lindholm
3,Par Lindholm,Roope Hintz
4,Blake Comeau,Patrice Bergeron


## Faceoff data for a `selected_player`

Get the Faceoff plays from the game data, and filter them down by a `selected_player`

- [ ] [TODO] Figure out iPython controls and make this a data-drive control (all players who took face-offs in the game)

In [26]:
select_player = 'Patrice Bergeron'

player_faceoffs = faceoff_df.loc[lambda p: (p['winner'] == select_player) | (p['loser'] == select_player)]

player_faceoffs.head(5)

,winner,loser
0,Patrice Bergeron,Tyler Seguin
4,Blake Comeau,Patrice Bergeron
6,Patrice Bergeron,Radek Faksa
9,Radek Faksa,Patrice Bergeron
12,Jamie Benn,Patrice Bergeron


## Faceoff Wins Count

Group by the winners and count the wins

In [27]:
player_faceoffs.groupby(['winner']).agg({'winner': 'count'})

,winner
winner,
Andrew Cogliano,1
Blake Comeau,1
Jamie Benn,3
Patrice Bergeron,8
Radek Faksa,3
Roope Hintz,1
Tyler Seguin,3
